In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('botswana.tsv', delimiter='\t')

In [3]:
df.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


In [4]:
df['religion'].unique().shape

(4,)

In [5]:
df.dropna().shape

(1834, 15)

In [6]:
def nevermar(row):
    if np.isnan(row['agefm']): 
        return 1 
    else: return 0

In [7]:
df['nevermar'] = df.apply(lambda row: nevermar(row), axis=1)

In [8]:
df.drop(columns=['evermarr'], inplace=True)

In [9]:
df['agefm'].fillna(0, inplace=True) #???

In [10]:
df['agefm'].isna().unique()

array([False])

In [11]:
def heduc(row):
    if row['nevermar'] == 1:
        return -1
    else:
        return row['heduc']

In [12]:
df['heduc'] = df.apply(lambda row: heduc(row), axis=1)

In [13]:
df['heduc'].value_counts(dropna=False)

-1.0     2282
 0.0      692
 7.0      396
 10.0     210
 NaN      123
 6.0      102
 12.0      99
 4.0       73
 3.0       64
 5.0       62
 2.0       47
 9.0       35
 15.0      35
 14.0      29
 16.0      26
 1.0       25
 17.0      17
 8.0       17
 11.0       9
 13.0       8
 18.0       4
 19.0       4
 20.0       2
Name: heduc, dtype: int64

In [31]:
dfd = df.copy()

In [38]:
dfd['idlnchld_noans'] = dfd['idlnchld'].isna().astype(int)
dfd['heduc_noans'] = dfd['heduc'].isna().astype(int)
dfd['idlnusemeth_noanschld_noans'] = dfd['usemeth'].isna().astype(int)

In [43]:
dfd['idlnchld'].fillna(-1, inplace=True)

In [47]:
dfd['heduc'].fillna(-2, inplace=True)

In [48]:
dfd['usemeth'].fillna(-1, inplace=True)

In [50]:
dfd['usemeth'].isna().unique()

array([False])

In [51]:
dfd.dropna(inplace=True)

In [52]:
dfd.index.shape[0] * dfd.columns.shape[0]

78264

**Regression**

In [53]:
import statsmodels.formula.api as smf

In [55]:
dfd.columns

Index(['ceb', 'age', 'educ', 'religion', 'idlnchld', 'knowmeth', 'usemeth',
       'agefm', 'heduc', 'urban', 'electric', 'radio', 'tv', 'bicycle',
       'nevermar', 'idlnchld_noans', 'heduc_noans',
       'idlnusemeth_noanschld_noans'],
      dtype='object')

In [85]:
formula_str = 'ceb ~'

for col in dfd.columns[1:]:
    if col == 'age':
        formula_str += ' '
    else:
        formula_str += ' + ' 
    formula_str += col

In [86]:
formula_str

'ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + radio + tv + bicycle + nevermar + idlnchld_noans + heduc_noans + idlnusemeth_noanschld_noans'

In [87]:
m1 = smf.ols(formula_str, data = dfd)

In [81]:
fitted = m1.fit()

In [82]:
fitted.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Tue, 03 Dec 2019   Prob (F-statistic):               0.00
Time:                        11:49:30   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                      -1.0263      0.212     -4.835      0.000      -1.443      -0.610
religion[T.other]              -0.0830      0.083     -1.001      0.317      -0.245       0.080
religion[T.protestant]         -0.0149      0.082     -0.181      0.857      -0.176       0.146
religion[T.spirit]             -0.0191      0.077     -0.248      0.804      -0.171       0.132
age                             0.1703      0.003     51.891      0.000       0.164       0.177
educ                           -0.0724      0.007     -9.843      0.000      -0.087      -0.058
idlnchld                        0.0760      0.011      6.923      0.000       0.054       0.098
knowmeth                        0.5564      0.121      4.580      0.000       0.318       0.795
usemeth                         0.6473      0.048     13.424      0.000       0.553       0.742
agefm                          -0.0604      0.007     -9.213      0.000      -0.073      -0.048
heduc                          -0.0551      0.008     -6.838      0.000      -0.071      -0.039
urban                          -0.2137      0.047     -4.527      0.000      -0.306      -0.121
electric                       -0.2685      0.077     -3.479      0.001      -0.420      -0.117
radio                          -0.0235      0.051     -0.461      0.645      -0.123       0.076
tv                             -0.1451      0.093     -1.566      0.118      -0.327       0.037
bicycle                         0.2139      0.050      4.260      0.000       0.115       0.312
nevermar                       -2.2393      0.148    -15.143      0.000      -2.529      -1.949
idlnchld_noans                  0.6539      0.153      4.286      0.000       0.355       0.953
heduc_noans                    -0.8724      0.145     -6.026      0.000      -1.156      -0.589
idlnusemeth_noanschld_noans     0.7652      0.196      3.910      0.000       0.382       1.149
==============================================================================
Omnibus:                      224.411   Durbin-Watson:                   1.887
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              859.014
Skew:                           0.003   Prob(JB):                    2.93e-187
Kurtosis:                       5.178   Cond. No.                         361.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [88]:
import statsmodels.stats.api as sms

In [93]:
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted.resid, fitted.model.exog)[1])

Breusch-Pagan test: p=0.000000


**Model without radio, religion, tv**

In [94]:
formula_str2 = 'ceb ~ age + educ + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + bicycle + nevermar + idlnchld_noans + heduc_noans + idlnusemeth_noanschld_noans'

In [95]:
m2 = smf.ols(formula_str2, data = dfd)

In [96]:
fitted2 = m2.fit()

In [97]:
fitted2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     559.5
Date:                Wed, 04 Dec 2019   Prob (F-statistic):               0.00
Time:                        23:13:54   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                      -1.0698      0.198     -5.393      0.000      -1.459      -0.681
age                             0.1702      0.003     52.271      0.000       0.164       0.177
educ                           -0.0729      0.007    -10.285      0.000      -0.087      -0.059
idlnchld                        0.0770      0.011      7.042      0.000       0.056       0.098
knowmeth                        0.5610      0.121      4.628      0.000       0.323       0.799
usemeth                         0.6516      0.048     13.537      0.000       0.557       0.746
agefm                          -0.0606      0.007     -9.240      0.000      -0.073      -0.048
heduc                          -0.0573      0.008     -7.186      0.000      -0.073      -0.042
urban                          -0.2190      0.047     -4.682      0.000      -0.311      -0.127
electric                       -0.3207      0.070     -4.584      0.000      -0.458      -0.184
bicycle                         0.2046      0.049      4.154      0.000       0.108       0.301
nevermar                       -2.2501      0.148    -15.231      0.000      -2.540      -1.961
idlnchld_noans                  0.6565      0.152      4.310      0.000       0.358       0.955
heduc_noans                    -0.8853      0.145     -6.122      0.000      -1.169      -0.602
idlnusemeth_noanschld_noans     0.7732      0.196      3.955      0.000       0.390       1.156
==============================================================================
Omnibus:                      224.096   Durbin-Watson:                   1.886
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              856.760
Skew:                           0.004   Prob(JB):                    9.06e-187
Kurtosis:                       5.175   Cond. No.                         345.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [103]:
round(fitted.compare_f_test(fitted2)[1],4)
#fitted.compare_f_test(fitted2)

0.4672

высокий уровень значимости -> оставляем модель без признаков

**Model without usemeth**

In [117]:
formula_str3 = 'ceb ~ age + educ + idlnchld + knowmeth + agefm + heduc + urban + electric + bicycle + nevermar + idlnchld_noans + heduc_noans'

In [118]:
m3 = smf.ols(formula_str3, data = dfd)

In [119]:
fitted3 = m3.fit()

In [124]:
#fitted3.summary()

In [121]:
#print("F=%f, p=%f, k1=%f" % fitted2.compare_f_test(fitted3))
fitted2.compare_f_test(fitted3)[1]

3.155200948040263e-40

низкий уровень значимости -> оставляем модель fitted2

In [127]:
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted2.resid, fitted2.model.exog)[1])

Breusch-Pagan test: p=0.000000


Ошибки гетероскедастичны, значит, значимость признаков может определяться неверно. Сделаем поправку Уайта:

In [128]:
fitted2_hc1 = m2.fit(cov_type='HC1')

In [129]:
fitted2_hc1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Wed, 04 Dec 2019   Prob (F-statistic):               0.00
Time:                        23:33:51   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
===============================================================================================
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                      -1.0698      0.258     -4.152      0.000      -1.575      -0.565
age                             0.1702      0.004     38.746      0.000       0.162       0.179
educ                           -0.0729      0.007    -10.311      0.000      -0.087      -0.059
idlnchld                        0.0770      0.014      5.323      0.000       0.049       0.105
knowmeth                        0.5610      0.174      3.224      0.001       0.220       0.902
usemeth                         0.6516      0.052     12.571      0.000       0.550       0.753
agefm                          -0.0606      0.010     -6.192      0.000      -0.080      -0.041
heduc                          -0.0573      0.009     -6.440      0.000      -0.075      -0.040
urban                          -0.2190      0.045     -4.814      0.000      -0.308      -0.130
electric                       -0.3207      0.063     -5.076      0.000      -0.445      -0.197
bicycle                         0.2046      0.048      4.279      0.000       0.111       0.298
nevermar                       -2.2501      0.202    -11.158      0.000      -2.645      -1.855
idlnchld_noans                  0.6565      0.216      3.043      0.002       0.234       1.079
heduc_noans                    -0.8853      0.191     -4.638      0.000      -1.259      -0.511
idlnusemeth_noanschld_noans     0.7732      0.212      3.639      0.000       0.357       1.190
==============================================================================
Omnibus:                      224.096   Durbin-Watson:                   1.886
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              856.760
Skew:                           0.004   Prob(JB):                    9.06e-187
Kurtosis:                       5.175   Cond. No.                         345.
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC1)
"""